# Schedule for one night with respect to objects priority.

## Libraries

In [ ]:
#setting up the libraries and packages to work with
import numpy as np #here we abbreviate "numpy" as "np" because we will use it with its commands like "np.command"
import pandas as pd
from pandas_ods_reader import read_ods
from astropy.time import Time
# import astroplan
from astroplan import Observer
from astropy.coordinates import EarthLocation
from astropy.table import Table
from astropy.coordinates import SkyCoord
import astropy.units as u
from pytz import timezone
from tabulate import tabulate

from astropy.utils.iers import conf
conf.auto_max_age = None

## Constant part

### Location

In [ ]:
longitude_tshao = '76d58m17.00s'
latitude_tshao = '43d03m29.00s'
elevation_tshao = 2735 * u.m
location_tshao = EarthLocation.from_geodetic(longitude_tshao, latitude_tshao, elevation_tshao)

tshao = Observer(name='tshao',
               location=location_tshao,
               pressure=0.615 * u.bar,
               relative_humidity=0.7,
               temperature=0 * u.deg_C,
               timezone=timezone('Asia/Almaty'),
               description="Tien Shan Astronomical Observatory, Kazaknstan")
# constraints = [AltitudeConstraint(18*u.deg, 80*u.deg), AirmassConstraint(max=3), AtNightConstraint.twilight_astronomical()]
lambd = (76+58/60+17/3600)*24/360
phi = (43 + 3/60 + 23/3600)*u.degree

### Tech time

In [ ]:
# Read-out time
readout_1bin = 20 * u.second
readout_2bin = 14 * u.second
readout_3bin = 10 * u.second

# Filter block turn time
fil_turn_near = 5 * u.second
fil_turn_far = 10 * u.second

# Telescope turning speed
slew_rate = .8*u.deg/u.second 

### Functions

In [ ]:
def sunset_rise_time(date):
    
    """
    Calculate sunset and surise time (astronomical twilight). 
    Also, calculate observation start and end SIDEREAL time    
    """
    
    # Sidereal time at midnight of the date
    
    JD1 = date.jd     #date in JD to future calculations
    T2_0 = (JD1 - 2433282.5) / 36524.2    #date in fraction of 100 years
    S0_2_0 = (6 + (40 / 60) + ((18.130 / 60) / 60)) + (8640184.635 /60 / 60 * T2_0) + (((0.0929 / 60) / 60) * T2_0**2) #sidereal time in Greenwich in UT midnight
    S0_2_0 = S0_2_0 % 24 #to account for only 24 hours in a day
    s_0 = S0_2_0 + lambd #local sidereal time in UT midnight
    s_0 = s_0%24 #to account for only 24 hours in a day
    sid_h_0 = s_0 - 6 #local sidereal time in local midnight

    #Sunset and sunrise local time and delta (midnight - SSLT, SRLT - midnight) 

    sunset_tonight = tshao.sun_set_time(date,  horizon = -18*u.degree, which='nearest') #+ 6*u.hour
    delta_set = (JD1 - (sunset_tonight+6*u.hour).value) * 24 #+ 6*u.hour because before we calculated time in UT, but now we need local

    sunrise_tonight = tshao.sun_rise_time(date,  horizon = -18*u.degree, which='nearest')  #+ 6*u.hour
    delta_rise = ((sunrise_tonight+6*u.hour).value - JD1) * 24 #+ 6*u.hour because before we calculated time in UT, but now we need local

    #Sunset and sunrise local sidereal time (same with strart and end of observation)

    sid_start = (sid_h_0 - delta_set)%24
    sid_end = (sid_h_0 + delta_rise) % 24
    
    time_range = [sunset_tonight, sunrise_tonight]

    return sunset_tonight, sunrise_tonight, sid_start, sid_end, time_range, sid_h_0


def target_obs_list(sid_start,sid_end):    
    
    """
    Check if RA of an object is in interval [sid_start,sid_end]. 
    In other words, could we observe an object culmination at night.
    Here we should respect Declanation, because some objects may be unvisible in our location.
    Moreovere, we can observe object 1 hour before and after culmination, so it is necessary to take into account.
    """
    
    names = [] #we should create list with obj names because we will write each object once. 
    alt = []
    
    for index in range(0,len(data_dec['RA'])):
        if data_dec['DEC'][index] > -30:
            
            h_ind = 90*u.degree + phi - data_dec['DEC'][index]*u.degree
            if h_ind > 90*u.degree:
                h_ind = 180*u.degree - h_ind
                
            if h_ind > 18*u.degree and h_ind < 80*u.degree: #if < 18, then airmass > 3
                ra_h = data_dec['RA'][index] * 24 / 360
                if sid_start < sid_end:
                    if (ra_h > sid_start and ra_h < sid_end): 
                        if data_dec['name'][index] not in names:
                            names.append(data_dec['name'][index])
                            alt.append(h_ind)
                else:
                    if (ra_h < sid_end or ra_h > sid_start):
                        if data_dec['name'][index] not in names:
                                names.append(data_dec['name'][index])
                                alt.append(h_ind)
            elif h_ind < 18*u.degree:
                continue
            
    return names, alt


def parking_coordinates(phi,date):

    """
    Find equatorial coordinates (RA,DEC) of telescope parking point.
    As we know, telescopes are parked usually on the South and paralell to the graund.
    So we have horizontal coordinates: azimut A = 270 deg and height  h = 0 deg.
    We need only two variable: lattitude of observatory (phi) and date of observations.
    """
    
    A = 180*u.degree
    z = 0*u.degree

    delta_rad = np.arcsin(np.sin(phi) * np.cos(z) - np.cos(phi) * np.sin(z) * np.cos(A))#* 360 / (2 * 3.14159265)
    parking_delta_deg = delta_rad.to(u.degree)

    sunset = tshao.sun_set_time(date,  horizon = -18*u.degree, which='nearest')
    JD2 = sunset.jd #date in JD to future calculations
    T = (JD2 - 2433282.5) / 36524.2    #date in fraction of 100 years
    S0 = (6 + (40 / 60) + ((18.130 / 60) / 60)) + (8640184.635 /60 / 60 * T) + (((0.0929 / 60) / 60) * T**2) #sidereal time in Greenwich in UT 
    S0 = S0 % 24 #to account for only 24 hours in a day
    s = S0 + lambd #UT sidereal time in local area 
    s = s%24 #to account for only 24 hours in a day
    sid_h = s - 6 #local sidereal time in local area
    parking_alpha_deg = s * 360 / 24
    
    a_h = int(parking_alpha_deg * 24 / 360)
    a_m = int((parking_alpha_deg * 24 / 360 - a_h) * 60)
    a_s = round(((parking_alpha_deg * 24 / 360 - a_h) * 60 - a_m)*60,2)
    parking_alpha_hms = f'{a_h} {a_m} {a_s}'

    d_d = int(parking_delta_deg.value)
    d_m = int((parking_delta_deg.value - d_d) * 60)
    d_s = round(((parking_delta_deg.value - d_d) * 60 - d_m) * 60,2)
    parking_dec_dms = f'{d_d} {d_m} {d_s}'
    
    return parking_alpha_deg, parking_delta_deg, parking_alpha_hms, parking_dec_dms


def slew_time(ra_1,dec_1,ra_2,dec_2):
    
    """
    Find time to target the object from previous one.
    """
    ra_1 = ra_1*u.degree
    dec_1 = dec_1*u.degree
    ra_2 = ra_2*u.degree
    dec_2 = dec_2*u.degree
    
    dist = (np.arccos(np.sin(dec_1) * np.sin(dec_2) + np.cos(dec_1) * np.cos(dec_2) * np.cos(ra_1 - ra_2))).to(u.degree)
    slew_time = (dist / slew_rate).to_value() * u.second
    
    return slew_time

def time_calculation(table):
    
    """
    Colculation of each object start, end, culmination and whole observation time.
    """
    
    culmination_time = []
    obj_start = []
    obj_end = []
    delta_time = []
    time_observ = []

    for ind in range(1,len(df)):
        
        ra = df['ra'][ind]
        ra_0 = str(ra).split(' ')
        ra_h = float(ra_0[0]) + float(ra_0[1])/60 + float(ra_0[2])/3600
        delta = sid_h_0 - ra_h

        if abs(delta) < 12:
            delta_h = delta*u.hour
            culm = JD - delta_h - 6*u.hour
            culmination_time.append(culm)

        elif abs(delta) > 12:
            delta_h = (24 - abs(delta))*u.hour
            culm = JD - delta_h - 6*u.hour
            culmination_time.append(culm)

        slew_t = slew_time(df['RA'][ind],df['DEC'][ind],df['RA'][ind-1],df['DEC'][ind-1])

        if df['bin'][ind] == 1:
            time = (df['B'][ind]*u.second + df['V'][ind]*u.second + fil_turn_near * 2 + df['R'][ind]*u.second + readout_1bin * 3) * df['n'][ind]#+ fil_turn_far

        elif df['bin'][ind] == 2: 
            time = (df['B'][ind]*u.second + df['V'][ind]*u.second + fil_turn_near * 2 + df['R'][ind]*u.second  + readout_2bin * 3) * df['n'][ind]#+ fil_turn_far

        elif df['bin'][ind] == 3:
            time = (df['B'][ind]*u.second + df['V'][ind]*u.second + fil_turn_near * 2 + df['R'][ind]*u.second  + readout_3bin * 3) * df['n'][ind]#+ fil_turn_far

        delta_t = time / 2
        delta_time.append(delta_t) 
        time_observ.append(time) 

        start = culm - delta_t - slew_t
        obj_start.append(start)

        end = culm + delta_t
        obj_end.append(end)

    return time_observ, obj_start, obj_end, culmination_time


def overlap_check(table):
    """
    Table with objects which WILL BE observed.
    Check if there overlap in time of obj observation.
    Sorting by priority.
    """
    prior_table = table
    drop_ind = []
    for ind in range(1,len(table_schedule)-1):
        """
        everything is ok, observations step by step
        """
        try:
            if prior_table['start'][ind] > prior_table['end'][ind-1]:
                continue

                """
                overlap in time
                """
            else:

                """
                TIME SHIFT IF IND > 2
                """
                if ind > 1: #we calculate ind-2
                    delta = prior_table['start'][ind-1] - prior_table['end'][ind-2]

                    if delta != 0*u.hour and delta < 2*u.hour:
                        prior_table['start'][ind-1] = prior_table['end'][ind-2]
                        prior_table['end'][ind-1] = prior_table['start'][ind-1] + prior_table['time_obs'][ind-1]
                    elif delta != 0*u.hour and delta > 2*u.hour:
                        prior_table['start'][ind-1] = prior_table['start'][ind-1] - 2*u.hour
                        prior_table['end'][ind-1] = prior_table['start'][ind-1] + prior_table['time_obs'][ind-1]
                        
                    """
                    TIME SHIFT FOR 2D OBJECT (shift 1st object to sunset)
                    """
                else: #overlap for the 2d object ind = 1
                    delta_sunset = prior_table['start'][ind-1] - sunset_tonight
                    if delta_sunset != 0*u.hour and delta_sunset < 2*u.hour:
                        prior_table['start'][ind-1] = sunset_tonight
                        prior_table['end'][ind-1] = prior_table['start'][ind-1] + prior_table['time_obs'][ind-1]
                    elif delta_sunset != 0*u.hour and delta_sunset > 2*u.hour:
                        prior_table['start'][ind-1] = prior_table['start'][ind-1] - 2*u.hour
                        prior_table['end'][ind-1] = prior_table['start'][ind-1] + prior_table['time_obs'][ind-1]
                    else:
                        prior_table['start'][ind-1] = prior_table['start'][ind-1]
                        prior_table['end'][ind-1] = prior_table['start'][ind-1]

                    """
                    CHECK OVERLAP AGAIN
                    """        
                if prior_table['start'][ind] > prior_table['end'][ind-1]:
                    continue

                else:
                    overlap = prior_table['end'][ind-1] - prior_table['start'][ind]
                    if overlap < (prior_table['time_obs'][ind]/4):
                        prior_table['start'][ind] = prior_table['start'][ind] + overlap
                        prior_table['end'][ind] = prior_table['end'][ind] + overlap
                    else:
                        m = int(prior_table['n'][ind - 1])
                        while m > 3:
                            m -= 1
                            obs_time_1 = df['B'][ind-1]*u.second + df['V'][ind-1]*u.second + fil_turn_near * 2 + df['R'][ind-1]*u.second + fil_turn_far + readout_3bin * 3
                            prior_table['end'][ind-1] = prior_table['end'][ind-1] - obs_time_1
                            overlap_m = prior_table['end'][ind-1] - prior_table['start'][ind]
                            #N - 1 worked
                            if overlap_m < (prior_table['time_obs'][ind]/4):
                                prior_table['n'][ind - 1] = m
                                prior_table['start'][ind] = prior_table['start'][ind] + overlap_m
                                prior_table['end'][ind] = prior_table['end'][ind] + overlap_m
                                break
                            #N - 1 didn't work. Repeat
                            else:
                                continue

                        # N < 3, we can't observe less than 3 series
                        # so we should take into accaunt priority
                        else:
                            """
                            Check percent of observation which were made
                            """
                            if float(prior_table['N_obs_perc'][ind-1]) > float(prior_table['N_obs_perc'][ind]):
                                drop_ind.append(ind)
                            elif float(prior_table['N_obs_perc'][ind-1]) < float(prior_table['N_obs_perc'][ind]):
                                drop_ind.append(ind-1)
                            else:
                                """
                                If percent of observation which were made is equal.
                                Check how many night of observations we need.
                                """
                                if float(prior_table['M'][ind-1]) > float(prior_table['M'][ind]):
                                    drop_ind.append(ind)
                                elif float(prior_table['M'][ind-1]) < float(prior_table['M'][ind]):
                                    drop_ind.append(ind-1)
                                else:
                                    """
                                    If percent of observation which were made is equal.
                                    If amount of neccesary observational nights is equal.
                                    Selecting an earlier object.
                                    """
                                    if float(prior_table['start'][ind-1]) > float(prior_table['start'][ind]): 
                                            drop_ind.append(ind)
                                    elif float(prior_table['start'][ind-1]) < float(prior_table['start'][ind]):
                                        drop_ind.append(ind-1)

        except TypeError: #because we have '-' in start column of END row
            continue
            
            """
            DROP objects, which will not observe this night
            """
    prior_table = prior_table.drop(labels = drop_ind).reset_index(drop=True)

    """
    Check if observations is duiring after sunrise
    """
    if prior_table['end'].iloc[len(prior_table)-2] > sunrise_tonight:
        slew_end_parking_overlap = slew_time(prior_table['RA_deg'].iloc[len(prior_table)-1], prior_table['DEC_deg'].iloc[len(prior_table)-1], prior_table['RA_deg'].iloc[len(prior_table)-3], prior_table['DEC_deg'].iloc[len(prior_table)-3]) 
        prior_table['end'].iloc[len(prior_table)-1] = prior_table['end'].iloc[len(prior_table)-3] + slew_end_parking_overlap
        prior_table = prior_table.drop(labels=len(prior_table)-2, axis=0)
        
        """
        Make new table with time without date.
        """
    prior_table_splitted = prior_table
    for t in range(0,len(prior_table_splitted)-1):
        prior_table_splitted['start'][t] = prior_table_splitted['start'][t].iso.split(" ")[1]
        prior_table_splitted['end'][t] = prior_table_splitted['end'][t].iso.split(" ")[1]
        prior_table_splitted['culmination'][t] = prior_table_splitted['culmination'][t].iso.split(" ")[1]
    prior_table_splitted['end'].iloc[-1] = prior_table_splitted['end'].iloc[-1].iso.split(" ")[1]
    
    prior_table_splitted = prior_table_splitted.reset_index(drop=True)
    
    return prior_table_splitted

## Variable part

### Date

In [ ]:
JD = Time('2021-02-19 00:00:00') #date of observations + 1 (because we need midnight)
#if we well make observations in night between 14th and 15th of February, we should write '2022-02-15 00:00:00'

### Import data

In [ ]:
data = read_ods('objects_example_test.ods')

data['RA'] = ''*len(data)
data['DEC'] = ''*len(data)
data['N'] = data['n']*data['M']

for ind,ra in enumerate(data['ra']):
    dec = str(data['dec'][ind]).split(' ')
    ra_0 = str(ra).split(' ')
    ra_h = float(ra_0[0]) + float(ra_0[1])/60 + float(ra_0[2])/3600
    data['RA'][ind] = ra_h * 360 / 24 #recalculate RA from hours to degrees     
    data['DEC'][ind] = float(dec[0]) + float(dec[1])/60 + float(dec[2])/3600
    
data_dec = data[data['DEC'].apply(lambda x: x > -30)]
data_dec = data_dec.reset_index(drop = True)

data_dec = data_dec.sort_values('RA') #because RA connected with culmination time and in the future staeps we will need in sorted by culm time list

del data

### Calculations

In [ ]:
#Calculate sunset, sunrise in local and local sidereal time.
sunset_tonight, sunrise_tonight, sid_start, sid_end, time_range, sid_h_0 = sunset_rise_time(JD)

#Let's make two different lists with observable targets (FixedTarget) and their name
names,altitude = target_obs_list(sid_start, sid_end)

#make table with only observable objects
data_obs = data_dec[data_dec['name'].apply(lambda name: name in names)].reset_index(drop=True)
data_obs['Alt'] = altitude
#Find equatorial coordinates (RA,DEC) of telescope parking point.
parking_alpha_deg, parking_delta_deg, parking_alpha_hms, parking_dec_dms = parking_coordinates(phi,JD)

# Parking point row. We should add it take into account slew time at the beginning and end of observation 
new_row = pd.DataFrame({'name':'parking_point', 'ra':parking_alpha_deg, 'dec':parking_delta_deg, 'B':0, 'V':0, 'R':0, 'bin':1, 'n':0, 'priority':0, 'M':0, 'N_obs_perc': 0,'RA':parking_alpha_deg, 'DEC':parking_delta_deg, 'N':0, 'Alt':0 }, index =[0])
df = pd.concat([new_row, data_obs]).reset_index(drop = True)

del data_dec
del data_obs

time_observ, obj_start, obj_end, culmination_time = time_calculation(df)

### Create a table with objects and their observing time 

In [ ]:
slew_end_parking = slew_time(df['RA'].iloc[0], df['DEC'].iloc[0], df['RA'].iloc[-1], df['DEC'].iloc[-1])    
obj_end.append((Time(obj_end[-1])+slew_end_parking))
obj_start.append('-')
culmination_time.append('-')
objects = np.append(np.array(df['name'][1:].apply(lambda x: str(x).replace(u'\xa0', u''))),'END')
ra = np.append(np.array(df['ra'][1:].apply(lambda x: str(x).replace(u'\xa0', u''))),parking_alpha_hms)
dec = np.append(np.array(df['dec'][1:].apply(lambda x: str(x).replace(u'\xa0', u''))),parking_dec_dms)
ra_deg = np.append(np.array(df['RA'][1:].apply(lambda x: float(str(x).replace(u'\xa0', u'')))),df['RA'].iloc[0])
dec_deg = np.append(np.array(df['DEC'][1:].apply(lambda x: float(str(x).replace(u'\xa0', u'')))), df['DEC'].iloc[0])
N = np.append(np.array(df['N'][1:]), 0)
B = np.append(np.array(df['B'][1:]), 0)
V = np.append(np.array(df['V'][1:]), 0)                        
R = np.append(np.array(df['R'][1:]), 0)                        
binning = np.append(np.array(df['bin'][1:]),0)
priority = np.append(np.array(df['priority'][1:]), 0)   
N_obs_perc = np.append(np.array(df['N_obs_perc'][1:]), 0) 
N_obs_perc = np.append(np.array(df['N_obs_perc'][1:]), 0) 
n = np.append(np.array(df['n'][1:]), 0)
M = np.append(np.array(df['M'][1:]), 0)
altitude = np.append(np.array(df['Alt'][1:].apply(lambda x: round(x.value,1))), 0)

time_observ.append(0*u.second)
              
table_schedule = pd.DataFrame({'name':objects, 'RA':ra, 'DEC':dec, 'RA_deg':ra_deg, 'DEC_deg':dec_deg, 'B':B,'V':V,'R':R, 'bin':binning, 'N':N, 'priority':priority,'start':obj_start, 'culmination':culmination_time, 'end':obj_end, 'time_obs':time_observ, 'n':n, 'Alt':altitude, 'M':M, 'N_obs_perc':N_obs_perc})

del objects; del ra; del dec

### Overlap check

In [ ]:
prior_table_spl = overlap_check(table_schedule)

### Change priority of observed objects

In [ ]:
prior_table_spl['N_obs_perc'] = round(prior_table_spl['n']/prior_table_spl['N'],2)
prior_table_spl['priority'] = round(prior_table_spl['priority'] - prior_table_spl['N_obs_perc'],2)

### New table with all objects and their new priority

In [ ]:
table_new = read_ods('objects_example_test.ods')

In [ ]:
counter = 0
for j in range(0,len(prior_table_spl)):
    for i in range(0,len(table_new)):
        if table_new['name'].iloc[i].replace("\xa0","") == prior_table_spl['name'].iloc[j]:
            counter+=1
            table_new['priority'].iloc[i] = prior_table_spl['priority'].iloc[j]
            table_new['N_obs_perc'].iloc[i] = prior_table_spl['N_obs_perc'].iloc[j]

## tabulating in LaTeX format

In [ ]:
print(tabulate(prior_table_spl, headers=prior_table_spl.columns, tablefmt="latex"))

## Save tables

In [ ]:
#Save table with plan at night
d = JD.value.split(" ")[0]

prior_table_spl.to_excel(f'{d}_schedule.xlsx',index=False)
#Save table with all objects
table_new.to_excel(f'objs_new_prior.xlsx',index=False)